In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn import cross_validation
import xgboost as xgb

def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [3]:
train = pd.read_csv('train_nnew.csv', low_memory=False)
test = pd.read_csv('test_nnew.csv', low_memory=False)
features = [u'Open', u'Promo', u'SchoolHoliday', u'StateHoliday_0',
       u'StateHoliday_a', u'DayOfWeek_1', u'DayOfWeek_2', u'DayOfWeek_3',
       u'DayOfWeek_4', u'DayOfWeek_5', u'DayOfWeek_6', u'DayOfWeek_7',
       u'CompetitionDistance', u'Promo2', 'year', 'Mean_Sales', 'month', 'day',
       u'StoreType_a', u'StoreType_b', u'StoreType_c', u'StoreType_d',
       u'Assortment_a', u'Assortment_b', u'Assortment_c', u'CompetitionOpen']

In [4]:
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(float)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(float)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(float)

In [5]:
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(float)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(float)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(float)

In [6]:
train.columns = [u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'StateHoliday0',
       u'StateHolidaya', u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3',
       u'DayOfWeek4', u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7',
       u'CompetitionDistance', u'Promo2', u'CompetitionOpenSince',
       u'StoreTypea', u'StoreTypeb', u'StoreTypec', u'StoreTyped',
       u'Assortmenta', u'Assortmentb', u'Assortmentc', u'CompetitionOpen',
       u'MeanSales', u'year', u'month', u'day']

In [7]:
test.columns = [u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'StateHoliday', u'SchoolHoliday', u'StateHolidaya', u'StateHoliday0',
       u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3', u'DayOfWeek4',
       u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7', u'CompetitionDistance',
       u'Promo2', u'CompetitionOpenSince', u'StoreTypea', u'StoreTypeb',
       u'StoreTypec', u'StoreTyped', u'Assortmenta', u'Assortmentb',
       u'Assortmentc', u'CompetitionOpen', u'MeanSales', u'year', u'month',
       u'day']

In [14]:
features = [u'Open', u'Promo', u'SchoolHoliday', u'StateHoliday0',
       u'StateHolidaya', u'DayOfWeek1', u'DayOfWeek2', u'DayOfWeek3',
       u'DayOfWeek4', u'DayOfWeek5', u'DayOfWeek6', u'DayOfWeek7',
       u'CompetitionDistance', u'Promo2', 'year', 'MeanSales', 'month', 'day',
       u'StoreTypea', u'StoreTypeb', u'StoreTypec', u'StoreTyped',
       u'Assortmenta', u'Assortmentb', u'Assortmentc', u'CompetitionOpen']

In [9]:
train['Date'] = train['Date'].astype('datetime64')
test['Date'] = test['Date'].astype('datetime64')

In [28]:
train_x = train[train['Store'] == 1]

In [29]:
train_x = train_x.sort(['Date'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [33]:
train_x.index = np.arange(len(train_x.index))

In [10]:
stores = np.unique(test.Store)

In [53]:
stores = [1]

In [62]:
store = 1

In [63]:
train_x = train[train['Store'] == store]
train_x = train_x.sort(['Date'])
train_x.index = np.arange(len(train_x.index))
test_x = test[test['Store'] == store]
test_x = test_x.sort(['Date'])
test_x.index = np.arange(len(test_x.index))
train_7 = train_x.copy()
train_18 = train_x.copy()
train_30 = train_x.copy()
test_7 = test_x.copy()
test_18 = test_x.copy()
test_30 = test_x.copy()
for step in xrange(6, len(train_7) - 2):
    train_7.loc[step-6,:] = train_x.loc[step-6:step].mean(axis=0)
    train_7.loc[step-6,'Sales'] = train_x.loc[step+1,'Sales']
    train_7.loc[step-6,'Open'] = train_x.loc[step+1,'Open']
train_7 = train_7.loc[6:,:]
for step in xrange(len(test_7)):
    s = min(step-6,0)
    test_7.loc[step,:] = test_x.loc[s:step].mean(axis=0)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
params = {"objective": "reg:linear", 
          "eta": 0.2, 
          "max_depth": 8, 
          "subsample": 0.9, 
          "colsample_bytree": 0.9, 
          "silent": 1, 
          "lambda" : 100, 
          "alpha" : 1
         } 
num_trees = 800

In [12]:
ans = test[['Id']]

In [ ]:
for store in stores:
    print 'store num', store
    train_x = train[train['Store'] == store]
    train_x = train_x.sort(['Date'])
    train_x.index = np.arange(len(train_x.index))
    test_x = test[test['Store'] == store]
    test_x = test_x.sort(['Date'])
    test_x.index = np.arange(len(test_x.index))
    train_7 = train_x.copy()
    train_18 = train_x.copy()
    train_30 = train_x.copy()
    test_7 = test_x.copy()
    test_18 = test_x.copy()
    test_30 = test_x.copy()
    for step in xrange(6, len(train_7) - 2):
        train_7.loc[step-6,:] = train_x.loc[step-6:step,:].mean(axis=0)
        train_7.loc[step-6,'Sales'] = train_x.loc[step+1,'Sales']
        train_7.loc[step-6,'Open'] = train_x.loc[step+1,'Open']
    train_7 = train_7.loc[6:,:]
    for step in xrange(len(test_7)):
        s = min(step-7,0)
        st = max(step-1, 0)
        test_7.loc[step,:] = test_x.loc[s:st,:].mean(axis=0)
        test_7.loc[step,'Id'] = test_x.loc[step,'Id']
        
    X_train, X_test = cross_validation.train_test_split(train_7, test_size=0.05, random_state = 1)
    dtrain = xgb.DMatrix(X_train[X_train['Open'] > 0][features], np.log(X_train[X_train['Open'] > 0]["Sales"] + 1))
    dvalid = xgb.DMatrix(X_test[X_test['Open'] > 0][features], np.log(X_test[X_test['Open'] > 0]["Sales"] + 1))
    dtest = xgb.DMatrix(test_7[features])
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, 
                    feval=rmspe_xg, verbose_eval=False)

    print("Validating")
    train_probs = gbm.predict(dvalid)
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[X_test['Open'] > 0]['Sales'].values)
    print('error', error)
    
    test_probs_7 = gbm.predict(dtest)
    indices = test_probs_7 < 0
    test_probs_7[indices] = 0
    
    for step in xrange(17, len(train_18) - 2):
        train_18.loc[step-17,:] = train_x.loc[step-17:step,:].mean(axis=0)
        train_18.loc[step-17,'Sales'] = train_x.loc[step+1,'Sales']
        train_18.loc[step-17,'Open'] = train_x.loc[step+1,'Open']
    train_18 = train_18.loc[17:,:]
    for step in xrange(len(test_18)):
        s = min(step-18,0)
        st = max(step-1, 0)
        test_18.loc[step,:] = test_x.loc[s:st,:].mean(axis=0)
        test_18.loc[step,'Id'] = test_x.loc[step,'Id']
    
    X_train, X_test = cross_validation.train_test_split(train_18, test_size=0.05, random_state = 1)
    dtrain = xgb.DMatrix(X_train[X_train['Open'] > 0][features], np.log(X_train[X_train['Open'] > 0]["Sales"] + 1))
    dvalid = xgb.DMatrix(X_test[X_test['Open'] > 0][features], np.log(X_test[X_test['Open'] > 0]["Sales"] + 1))
    dtest = xgb.DMatrix(test_18[features])
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, 
                    feval=rmspe_xg, verbose_eval=False)

    print("Validating")
    train_probs = gbm.predict(dvalid)
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[X_test['Open'] > 0]['Sales'].values)
    print('error', error)
    
    test_probs_18 = gbm.predict(dtest)
    indices = test_probs_18 < 0
    test_probs_18[indices] = 0
    
    for step in xrange(29, len(train_30) - 2):
        train_30.loc[step-29,:] = train_x.loc[step-29:step,:].mean(axis=0)
        train_30.loc[step-29,'Sales'] = train_x.loc[step+1,'Sales']
        train_30.loc[step-29,'Open'] = train_x.loc[step+1,'Open']
    train_30 = train_30.loc[29:,:]
    for step in xrange(len(test_30)):
        s = min(step-30,0)
        st = max(step-1, 0)
        test_30.loc[step,:] = test_x.loc[s:st,:].mean(axis=0)
        test_30.loc[step,'Id'] = test_x.loc[step,'Id']
    
    X_train, X_test = cross_validation.train_test_split(train_30, test_size=0.05, random_state = 1)
    dtrain = xgb.DMatrix(X_train[X_train['Open'] > 0][features], np.log(X_train[X_train['Open'] > 0]["Sales"] + 1))
    dvalid = xgb.DMatrix(X_test[X_test['Open'] > 0][features], np.log(X_test[X_test['Open'] > 0]["Sales"] + 1))
    dtest = xgb.DMatrix(test_30[features])
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, 
                    feval=rmspe_xg, verbose_eval=False)

    print("Validating")
    train_probs = gbm.predict(dvalid)
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[X_test['Open'] > 0]['Sales'].values)
    print('error', error)
    
    test_probs_30 = gbm.predict(dtest)
    indices = test_probs_30 < 0
    test_probs_30[indices] = 0
    
    ans.loc[test_7.Id-1, 'Sales_7'] = np.exp(test_probs_7) - 1
    ans.loc[test_18.Id-1, 'Sales_18'] = np.exp(test_probs_18) - 1
    ans.loc[test_30.Id-1, 'Sales_30'] = np.exp(test_probs_30) - 1
    ans['Sales'] = ans.loc[:, ['Sales_7','Sales_18','Sales_30']].mean(axis=1)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
Will train until train error hasn't decreased in 50 rounds.


store num 1
Validating
('error', 0.11149200467705904)
Validating

Will train until train error hasn't decreased in 50 rounds.
Will train until train error hasn't decreased in 50 rounds.



('error', 0.078929655726852191)
Validating

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f


('error', 0.12459262481361415)
store num 3
Validating

Will train until train error hasn't decreased in 50 rounds.
Will train until train error hasn't decreased in 50 rounds.



('error', 0.26670400465875854)
Validating
('error', 0.12088407180845552)
Validating
('error', 0.15130839102629839)
store num

Will train until train error hasn't decreased in 50 rounds.
Will train until train error hasn't decreased in 50 rounds.


 7
Validating

In [ ]:
print 'OLOLO'

In [ ]:
ans[['Id', 'Sales']].to_csv("xgb_ts_ind.csv", index=False)

In [55]:
len(train_x)

942

In [117]:
train_x.loc[0:0]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,2,2013-01-01,0,0,0,0,a,1,0,...,1,0,1,0,0,1,3835.285714,2013,1,1


In [36]:
train_x.loc[0:6]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,2,2013-01-01,0,0,0,0,a,1,0,...,1,0,1,0,0,1,3835.285714,2013,1,1
1,1,3,2013-01-02,5530,668,1,0,0,1,1,...,1,0,1,0,0,1,3746.746032,2013,1,2
2,1,4,2013-01-03,4327,578,1,0,0,1,1,...,1,0,1,0,0,1,3419.396825,2013,1,3
3,1,5,2013-01-04,4486,619,1,0,0,1,1,...,1,0,1,0,0,1,4206.063492,2013,1,4
4,1,6,2013-01-05,4997,635,1,0,0,1,1,...,1,0,1,0,0,1,4942.970149,2013,1,5
5,1,7,2013-01-06,0,0,0,0,0,1,1,...,1,0,1,0,0,1,0.000000,2013,1,6
6,1,1,2013-01-07,7176,785,1,1,0,1,1,...,1,0,1,0,0,1,5852.708333,2013,1,7


In [45]:
train_z = train_x.loc[0:6]

In [46]:
train_z.loc[0,:] = train_x.loc[0:6].mean(axis=0)

In [48]:
train_z.loc[1,:] = train_x.loc[1:7].mean(axis=0)

In [50]:
train_x.loc[1:7].mean(axis=0)

Store                     1.000000
DayOfWeek                 4.000000
Sales                  4585.142857
Customers               562.714286
Open                      0.857143
Promo                     0.285714
SchoolHoliday             1.000000
StateHoliday_0            1.000000
StateHoliday_a            0.000000
DayOfWeek_1               0.142857
DayOfWeek_2               0.142857
DayOfWeek_3               0.142857
DayOfWeek_4               0.142857
DayOfWeek_5               0.142857
DayOfWeek_6               0.142857
DayOfWeek_7               0.142857
CompetitionDistance    1270.000000
Promo2                    0.000000
StoreType_a               0.000000
StoreType_b               0.000000
StoreType_c               1.000000
StoreType_d               0.000000
Assortment_a              1.000000
Assortment_b              0.000000
Assortment_c              0.000000
CompetitionOpen           1.000000
Mean_Sales             3933.211722
year                   2013.000000
month               

In [49]:
train_z

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StateHoliday_0,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,CompetitionOpen,Mean_Sales,year,month,day
0,1,4,NaT,3788.000000,469.285714,0.714286,0.142857,NaN,1,0.857143,...,1,0,1,0,0,1,3697.517640,2013,1,4.428571
1,1,4,NaT,4585.142857,562.714286,0.857143,0.285714,NaN,1,1.000000,...,1,0,1,0,0,1,3933.211722,2013,1,5.000000
2,1,4,2013-01-03,4327.000000,578.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,3419.396825,2013,1,3.000000
3,1,5,2013-01-04,4486.000000,619.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,4206.063492,2013,1,4.000000
4,1,6,2013-01-05,4997.000000,635.000000,1.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,4942.970149,2013,1,5.000000
5,1,7,2013-01-06,0.000000,0.000000,0.000000,0.000000,0,1,1.000000,...,1,0,1,0,0,1,0.000000,2013,1,6.000000
6,1,1,2013-01-07,7176.000000,785.000000,1.000000,1.000000,0,1,1.000000,...,1,0,1,0,0,1,5852.708333,2013,1,7.000000
